In [1]:
%run data_processing.py
# memory leaking!

23-12-27 02:03:38|MOD:data_processing|: Data Processing start!
23-12-27 02:03:38|MOD:data_processing|: 3 datas :['y', 'trade_day', 'trade_15m']


Wed Dec 27 02:03:38 2023 : y start ...
y blocks reading 10days/lag1 Data1D's ...... cost 9.05 secs
y blocks reading 20days/lag1 Data1D's ...... cost 8.39 secs
y blocks merging ...... cost 4.10 secs
y blocks process ...... cost 0.55 secs
y blocks saving  ...... cost 18.51 secs
y blocks norming ...... cost 0.00 secs
Wed Dec 27 02:04:19 2023 : y finished! Cost 40.67 Seconds
Wed Dec 27 02:04:19 2023 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 46.29 secs
trade_day blocks merging ...... cost 0.12 secs
trade_day blocks process ...... cost 5.34 secs
trade_day blocks saving  ...... cost 25.05 secs
trade_day blocks norming ...... cost 34.22 secs
Wed Dec 27 02:06:10 2023 : trade_day finished! Cost 111.11 Seconds
Wed Dec 27 02:06:10 2023 : trade_15m start ...
trade_15m blocks reading 15min/trade Data1D's ...... cost 1045.64 secs
trade_15m blocks merging ...... cost 0.11 secs
trade_15m blocks process ...... cost 16.26 secs
trade_15m blocks saving  ...... cost 166.6

23-12-27 02:27:22|MOD:data_processing|: Data Processing Finished! Cost 1423.68 Seconds


... cost 42.68 secs
Wed Dec 27 02:27:21 2023 : trade_15m finished! Cost 1271.34 Seconds


In [1]:
from scripts.data_util.ModelData import *
new_block = DataBlock().read_npz('./data/block_data/X_trade_15m.npz')

In [1]:
from scripts.util.basic import timer
from scripts.data_util.ModelData import *
ThisBlock = DataBlock().read_npz('./data/block_data/X_trade_15m.npz')
key = 'trade_15m'

def save_norm_path(key):
    DIR_hist_norm = f'{DIR_data}/hist_norm'
    return f'{DIR_hist_norm}/X_{key}.npz'

with timer(f'norming {key} blocks') as t:
    block_hist_norm(ThisBlock , key , save_norm_path(key))

norming trade_15m blocks ...Used: 19.93G; Free 39.46G
Used: 28.20G; Free 31.19G
... cost 112.93 secs


In [5]:
a=torch.rand(100)
a

tensor([0.4191, 0.3421, 0.1349, 0.0498, 0.6775, 0.6405, 0.9351, 0.5546, 0.1751,
        0.7331, 0.5037, 0.0680, 0.2415, 0.9101, 0.1170, 0.4114, 0.6925, 0.1825,
        0.1131, 0.2004, 0.5198, 0.2730, 0.7980, 0.0962, 0.4550, 0.9462, 0.2056,
        0.3839, 0.6993, 0.9921, 0.9517, 0.5271, 0.7775, 0.4003, 0.3211, 0.1256,
        0.8574, 0.1852, 0.6136, 0.3629, 0.3919, 0.4521, 0.6654, 0.9781, 0.4850,
        0.0797, 0.7361, 0.7792, 0.4768, 0.6373, 0.9451, 0.2385, 0.3710, 0.5454,
        0.5720, 0.0517, 0.4701, 0.0324, 0.6156, 0.5682, 0.0250, 0.5079, 0.3114,
        0.0334, 0.6917, 0.1383, 0.6629, 0.7261, 0.3345, 0.5140, 0.4490, 0.9667,
        0.1192, 0.8801, 0.0168, 0.7882, 0.3569, 0.5201, 0.9235, 0.5178, 0.6674,
        0.0490, 0.4427, 0.2488, 0.1653, 0.8933, 0.1827, 0.5331, 0.2352, 0.4718,
        0.8585, 0.6016, 0.3742, 0.2429, 0.5568, 0.1290, 0.7793, 0.2681, 0.5947,
        0.8786])

In [4]:
a = 'a'
b = 'b'
import os
os.path.join(a,b) , '\\'.join([a,b])

('a/b', 'a\\b')

In [1]:
from scripts.data_util.ModelData import *
from scripts.util.environ import get_config
config = get_config()
md = ModelData(['day'] , config)

Load  2 DataBlocks ...... cost 5.96 secs
Align 2 DataBlocks ...... cost 6.29 secs


In [1]:
from scripts.data_util.ModelData import *
from scripts.util.environ import get_config
config = get_config()
md = ModelData(['day'] , config)

Load  2 DataBlocks ...... cost 0.00 secs
Align 2 DataBlocks ...... cost 12.15 secs


In [4]:
from scripts.data_util.ModelData import *
from scripts.util.environ import get_config
config = get_config()
md = ModelData(['day'] , config)

use /home/mengkjin/Workspace/learndl/scripts/util/../../data/torch_pack/day+rtn11+res11.pt


In [2]:
from scripts.data_util.DataTank import DataTank
import traceback
DB_name = 'data/DB_3rdPartySQL.h5'
try:
    dtank = DataTank(DB_name , True , 'r+')
    dtank.rewrite_all()
except:
    traceback.print_exc()
finally:
    dtank.close()

/
/dongfang
/dongfang/gp
/dongfang/gp/20170103
/dongfang/gp/20170104
/dongfang/gp/20170105
/dongfang/gp/20170106
/dongfang/gp/20170109
/dongfang/gp/20170110
/dongfang/gp/20170111
/dongfang/gp/20170112
/dongfang/gp/20170113
/dongfang/gp/20170116
/dongfang/gp/20170117
/dongfang/gp/20170118
/dongfang/gp/20170119
/dongfang/gp/20170120
/dongfang/gp/20170123
/dongfang/gp/20170124
/dongfang/gp/20170125
/dongfang/gp/20170126
/dongfang/gp/20170203
/dongfang/gp/20170206
/dongfang/gp/20170207
/dongfang/gp/20170208
/dongfang/gp/20170209
/dongfang/gp/20170210
/dongfang/gp/20170213
/dongfang/gp/20170214
/dongfang/gp/20170215
/dongfang/gp/20170216
/dongfang/gp/20170217
/dongfang/gp/20170220
/dongfang/gp/20170221
/dongfang/gp/20170222
/dongfang/gp/20170223
/dongfang/gp/20170224
/dongfang/gp/20170227
/dongfang/gp/20170228
/dongfang/gp/20170301
/dongfang/gp/20170302
/dongfang/gp/20170303
/dongfang/gp/20170306
/dongfang/gp/20170307
/dongfang/gp/20170308
/dongfang/gp/20170309
/dongfang/gp/20170310
/dongfa